In [11]:
myopt={
  "task":"swinir_sr_lightweight_x2"     
  , "model": "plain" 
  , "gpu_ids": [0]
  , "dist": False

  , "scale": 2       
  , "n_channels": 3  

  , "path": {
    "root": "superresolution"            
    , "pretrained_netG": None      
    , "pretrained_netE": None      
  }

  , "datasets": {
    "train": {
      "name": "train_dataset"          
      , "dataset_type": "sr"         
      , "dataroot_H": "trainsets/trainH"
      , "dataroot_L": "trainsets/trainL"  

      , "H_size": 128                   

      , "dataloader_shuffle": True
      , "dataloader_num_workers": 0
      , "dataloader_batch_size": 4      
    }
    , "test": {
      "name": "test_dataset"            
      , "dataset_type": "sr"         
      , "dataroot_H": "testsets/Set5/HR"  
      , "dataroot_L": "testsets/Set5/LR_bicubic/X2"   

    }
  }

  , "netG": {
    "net_type": "swinir" 
    , "upscale": 2                      
    , "in_chans": 3 
    , "img_size": 64
    , "window_size": 8  
    , "img_range": 1.0 
    , "depths": [6, 6, 6, 6]
    , "embed_dim": 60 
    , "num_heads": [6, 6, 6, 6]
    , "mlp_ratio": 2 
    , "upsampler": "pixelshuffledirect"       
    , "resi_connection": "1conv"       

    , "init_type": "default"
  }

  , "train": {
    "G_lossfn_type": "l1"               
    , "G_lossfn_weight": 1.0           

    , "E_decay": 0.999                  

    , "G_optimizer_type": "adam"       
    , "G_optimizer_lr": 2e-4            
    , "G_optimizer_wd": 0              
    , "G_optimizer_clipgrad": None      
    , "G_optimizer_reuse": True         

    , "G_scheduler_type": "MultiStepLR" 
    , "G_scheduler_milestones": [250000, 400000, 450000, 475000, 500000]
    , "G_scheduler_gamma": 0.5

    , "G_regularizer_orthstep": None    
    , "G_regularizer_clipstep": None    

    , "G_param_strict": True
    , "E_param_strict": True

    , "checkpoint_test":  4          
    , "checkpoint_save":  4          
    , "checkpoint_print": 2           
  }
    }


In [12]:
import os.path
import math
import argparse
import time
import random
import numpy as np
from collections import OrderedDict
import logging
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler
import torch

from utils import utils_logger
from utils import utils_image as util
from utils import utils_option as option
from utils.utils_dist import get_dist_info, init_dist

from data.select_dataset import define_Dataset
from models.select_model import define_Model

In [17]:
json_path='options/train_swinir_sr_lightweight.json'

In [21]:
opt = option.parse(json_path, is_train=True)
opt['dist'] = False

export CUDA_VISIBLE_DEVICES=0
number of GPUs is: 1


In [22]:
opt

OrderedDict([('task', 'swinir_sr_lightweight_x2'),
             ('model', 'plain'),
             ('gpu_ids', [0]),
             ('dist', False),
             ('scale', 2),
             ('n_channels', 3),
             ('path',
              OrderedDict([('root', 'superresolution'),
                           ('pretrained_netG', None),
                           ('pretrained_netE', None),
                           ('task',
                            'superresolution\\swinir_sr_lightweight_x2'),
                           ('log',
                            'superresolution\\swinir_sr_lightweight_x2'),
                           ('options',
                            'superresolution\\swinir_sr_lightweight_x2\\options'),
                           ('models',
                            'superresolution\\swinir_sr_lightweight_x2\\models'),
                           ('images',
                            'superresolution\\swinir_sr_lightweight_x2\\images')])),
             ('datasets',


In [32]:
opt['rank'], opt['world_size'] = get_dist_info()

if opt['rank'] == 0:
    util.mkdirs((path for key, path in opt['path'].items() if 'pretrained' not in key))

# update opt

# -->-->-->-->-->-->-->-->-->-->-->-->-->-
init_iter_G, init_path_G = option.find_last_checkpoint(opt['path']['models'], net_type='G')
init_iter_E, init_path_E = option.find_last_checkpoint(opt['path']['models'], net_type='E')
opt['path']['pretrained_netG'] = init_path_G
opt['path']['pretrained_netE'] = init_path_E
init_iter_optimizerG, init_path_optimizerG = option.find_last_checkpoint(opt['path']['models'], net_type='optimizerG')
opt['path']['pretrained_optimizerG'] = init_path_optimizerG
current_step = max(init_iter_G, init_iter_E, init_iter_optimizerG)

border = opt['scale']
# --<--<--<--<--<--<--<--<--<--<--<--<--<-
# ----------------------------------------
# save opt to  a '../option.json' file
# ----------------------------------------
if opt['rank'] == 0:
    option.save(opt)

# ----------------------------------------

# ----------------------------------------
# return None for missing key
# ----------------------------------------
opt = option.dict_to_nonedict(opt)

# ----------------------------------------
# configure logger
# ----------------------------------------
if opt['rank'] == 0:
    logger_name = 'train'
    utils_logger.logger_info(logger_name, os.path.join(opt['path']['log'], logger_name+'.log'))
    logger = logging.getLogger(logger_name)
    logger.info(option.dict2str(opt))

# ----------------------------------------
# seed
# ----------------------------------------
seed = opt['train']['manual_seed']
if seed is None:
    seed = random.randint(1, 10000)
print('Random seed: {}'.format(seed))
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)


22-12-11 15:12:06.684 :   task: swinir_sr_lightweight_x2
  model: plain
  gpu_ids: [0]
  dist: False
  scale: 2
  n_channels: 3
  path:[
    root: superresolution
    pretrained_netG: superresolution\swinir_sr_lightweight_x2\models\260_G.pth
    pretrained_netE: superresolution\swinir_sr_lightweight_x2\models\260_E.pth
    task: superresolution\swinir_sr_lightweight_x2
    log: superresolution\swinir_sr_lightweight_x2
    options: superresolution\swinir_sr_lightweight_x2\options
    models: superresolution\swinir_sr_lightweight_x2\models
    images: superresolution\swinir_sr_lightweight_x2\images
    pretrained_optimizerG: superresolution\swinir_sr_lightweight_x2\models\260_optimizerG.pth
  ]
  datasets:[
    train:[
      name: train_dataset
      dataset_type: sr
      dataroot_H: trainsets/trainH
      dataroot_L: trainsets/trainL
      H_size: 128
      dataloader_shuffle: True
      dataloader_num_workers: 0
      dataloader_batch_size: 4
      phase: train
      scale: 2
      n_

LogHandlers exist!
Random seed: 9378


In [33]:

    '''
    # ----------------------------------------
    # Step--2 (creat dataloader)
    # ----------------------------------------
    '''

    # ----------------------------------------
    # 1) create_dataset
    # 2) creat_dataloader for train and test
    # ----------------------------------------
    for phase, dataset_opt in opt['datasets'].items():
        if phase == 'train':
            train_set = define_Dataset(dataset_opt)
            train_size = int(math.ceil(len(train_set) / dataset_opt['dataloader_batch_size']))
            if opt['rank'] == 0:
                logger.info('Number of train images: {:,d}, iters: {:,d}'.format(len(train_set), train_size))
            if opt['dist']:
                train_sampler = DistributedSampler(train_set, shuffle=dataset_opt['dataloader_shuffle'], drop_last=True, seed=seed)
                train_loader = DataLoader(train_set,
                                          batch_size=dataset_opt['dataloader_batch_size']//opt['num_gpu'],
                                          shuffle=False,
                                          num_workers=dataset_opt['dataloader_num_workers']//opt['num_gpu'],
                                          drop_last=True,
                                          pin_memory=True,
                                          sampler=train_sampler)
            else:
                train_loader = DataLoader(train_set,
                                          batch_size=dataset_opt['dataloader_batch_size'],
                                          shuffle=dataset_opt['dataloader_shuffle'],
                                          num_workers=dataset_opt['dataloader_num_workers'],
                                          drop_last=True,
                                          pin_memory=True)

        elif phase == 'test':
            test_set = define_Dataset(dataset_opt)
            test_loader = DataLoader(test_set, batch_size=1,
                                     shuffle=False, num_workers=1,
                                     drop_last=False, pin_memory=True)
        else:
            raise NotImplementedError("Phase [%s] is not recognized." % phase)


22-12-11 15:12:42.197 : Number of train images: 10, iters: 3


Dataset [DatasetSR - train_dataset] is created.
Dataset [DatasetSR - test_dataset] is created.


In [34]:

'''
# ----------------------------------------
# Step--3 (initialize model)
# ----------------------------------------
'''

model = define_Model(opt)
model.init_train()
if opt['rank'] == 0:
    logger.info(model.info_network())
    logger.info(model.info_params())


D:\ProgramData\Anaconda\envs\swinir\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Pass this initialization! Initialization was done during network definition!
Pass this initialization! Initialization was done during network definition!
Training model [ModelPlain] is created.
Loading model for G [superresolution\swinir_sr_lightweight_x2\models\260_G.pth] ...
Loading model for E [superresolution\swinir_sr_lightweight_x2\models\260_E.pth] ...
Loading optimizerG [superresolution\swinir_sr_lightweight_x2\models\260_optimizerG.pth] ...


22-12-11 15:13:09.785 : 
Networks name: SwinIR
Params number: 910152
Net structure:
SwinIR(
  (conv_first): Conv2d(3, 60, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (patch_embed): PatchEmbed(
    (norm): LayerNorm((60,), eps=1e-05, elementwise_affine=True)
  )
  (patch_unembed): PatchUnEmbed()
  (pos_drop): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0): RSTB(
      (residual_group): BasicLayer(
        dim=60, input_resolution=(64, 64), depth=6
        (blocks): ModuleList(
          (0): SwinTransformerBlock(
            dim=60, input_resolution=(64, 64), num_heads=6, window_size=8, shift_size=0, mlp_ratio=2
            (norm1): LayerNorm((60,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              dim=60, window_size=(8, 8), num_heads=6
              (qkv): Linear(in_features=60, out_features=180, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=60, out_features=60, 

22-12-11 15:13:09.942 : 
 |  mean  |  min   |  max   |  std   || shape               
 | -0.000 | -0.198 |  0.196 |  0.111 | torch.Size([60, 3, 3, 3]) || conv_first.weight
 | -0.029 | -0.189 |  0.186 |  0.107 | torch.Size([60]) || conv_first.bias
 |  1.001 |  0.996 |  1.007 |  0.002 | torch.Size([60]) || patch_embed.norm.weight
 |  0.000 | -0.004 |  0.003 |  0.001 | torch.Size([60]) || patch_embed.norm.bias
 |  0.999 |  0.996 |  1.004 |  0.002 | torch.Size([60]) || layers.0.residual_group.blocks.0.norm1.weight
 | -0.000 | -0.005 |  0.004 |  0.002 | torch.Size([60]) || layers.0.residual_group.blocks.0.norm1.bias
 | -0.001 | -0.071 |  0.071 |  0.021 | torch.Size([225, 6]) || layers.0.residual_group.blocks.0.attn.relative_position_bias_table
 | 112.000 |  0.000 | 224.000 | 48.719 | torch.Size([64, 64]) || layers.0.residual_group.blocks.0.attn.relative_position_index
 |  0.000 | -0.083 |  0.074 |  0.020 | torch.Size([180, 60]) || layers.0.residual_group.blocks.0.attn.qkv.weight
 |  0.000 |

In [94]:

    '''
    # ----------------------------------------
    # Step--4 (main training)
    # ----------------------------------------
    '''

    logmyloss = torch.Tensor()
    logmypsnr = torch.Tensor()
    for epoch in range(10):  # keep running
        if opt['dist']:
            train_sampler.set_epoch(epoch)

        for i, train_data in enumerate(train_loader):

            current_step += 1

            # -------------------------------
            # 1) update learning rate
            # -------------------------------
            model.update_learning_rate(current_step)

            # -------------------------------
            # 2) feed patch pairs
            # -------------------------------
            model.feed_data(train_data)

            # -------------------------------
            # 3) optimize parameters
            # -------------------------------
            model.optimize_parameters(current_step)

            # -------------------------------
            # 4) training information
            # -------------------------------
            if current_step % opt['train']['checkpoint_print'] == 0 and opt['rank'] == 0:
                logs = model.current_log()  # such as loss
                message = '<epoch:{:3d}, iter:{:8,d}, lr:{:.3e}> '.format(epoch, current_step, model.current_learning_rate())
                for k, v in logs.items():  # merge log information into message
                    message += '{:s}: {:.3e} '.format(k, v)
                    global logmyloss
                    logmyloss = torch.cat([logmyloss, torch.tensor([v])])
                    torch.save(logmyloss, os.path.join("loss", 'last_loss.pth'))
                    
                logger.info(message)

            # -------------------------------
            # 5) save model
            # -------------------------------
            if current_step % opt['train']['checkpoint_save'] == 0 and opt['rank'] == 0:
                logger.info('Saving the model.')
                model.save(current_step)

            # -------------------------------
            # 6) testing
            # -------------------------------
            if current_step % opt['train']['checkpoint_test'] == 0 and opt['rank'] == 0:

                avg_psnr = 0.0
                idx = 0

                for test_data in test_loader:
                    idx += 1
                    image_name_ext = os.path.basename(test_data['L_path'][0])
                    img_name, ext = os.path.splitext(image_name_ext)

                    img_dir = os.path.join(opt['path']['images'], img_name)
                    util.mkdir(img_dir)
                    print(img_dir)

                    model.feed_data(test_data)
                    model.test()

                    visuals = model.current_visuals()
                    E_img = util.tensor2uint(visuals['E'])
                    H_img = util.tensor2uint(visuals['H'])

                    # -----------------------
                    # save estimated image E
                    # -----------------------
                    save_img_path = os.path.join(img_dir, '{:s}_{:d}.png'.format(img_name, current_step))
                    util.imsave(E_img, save_img_path)

                    # -----------------------
                    # calculate PSNR
                    # -----------------------
                    current_psnr = util.calculate_psnr(E_img, H_img, border=border)

                    logger.info('{:->4d}--> {:>10s} | {:<4.2f}dB'.format(idx, image_name_ext, current_psnr))

                    avg_psnr += current_psnr

                avg_psnr = avg_psnr / idx

                # testing log
                logger.info('<epoch:{:3d}, iter:{:8,d}, Average PSNR : {:<.2f}dB\n'.format(epoch, current_step, avg_psnr))


22-12-11 16:01:18.877 : <epoch:  0, iter:     332, lr:2.000e-04> G_loss: 3.097e-02 
22-12-11 16:01:18.878 : Saving the model.


superresolution\swinir_sr_lightweight_x2\images\baby


22-12-11 16:01:25.165 : ---1-->   baby.png | 30.36dB


superresolution\swinir_sr_lightweight_x2\images\bird


22-12-11 16:01:25.840 : ---2-->   bird.png | 29.05dB


...

22-12-11 16:02:30.902 : ---3--> butterfly.png | 24.99dB


superresolution\swinir_sr_lightweight_x2\images\head


22-12-11 16:02:31.544 : ---4-->   head.png | 29.32dB


superresolution\swinir_sr_lightweight_x2\images\woman


22-12-11 16:02:32.208 : ---5-->  woman.png | 29.26dB
22-12-11 16:02:32.499 : <epoch:  8, iter:     348, Average PSNR : 28.97dB

22-12-11 16:02:36.442 : <epoch:  9, iter:     350, lr:2.000e-04> G_loss: 3.525e-02 


In [88]:

c=torch.tensor(a)
array=[a]

In [89]:
array

[1.0000555]

In [92]:
a=1.0000555
c=torch.tensor([a])

In [79]:
c.type()

'torch.FloatTensor'

In [93]:
c.shape

torch.Size([1])

In [65]:
torch.from_numpy(a)

TypeError: expected np.ndarray (got float)

In [45]:
logs.items()

odict_items([('G_loss', 0.02739609405398369)])

In [72]:
for k,v in logs.items():
    print(v)
    a=torch.tensor(v)
    print(a)

0.030511628836393356
tensor(0.0305)


odict_items([('G_loss', 0.02739609405398369)])